In [ ]:
import vice

In [ ]:
import numpy as np

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import surp
from dataclasses import dataclass

import surp.gce_math as gcem
import arya

In [ ]:
import toml

In [ ]:
@dataclass
class MCMCResult:
    params: dict
    labels: list
    samples: pd.DataFrame
    afe: pd.DataFrame
    ah: pd.DataFrame


    @classmethod
    def from_file(cls, modelname):
        modeldir = "../models/perturbations/mc_analysis/" + modelname + "/"
    
        with open(modeldir + "params.toml", "r") as f:
            params = toml.load(f)
    
    
        samples = pd.read_csv(modeldir + "mcmc_samples.csv")
        afe = pd.read_csv(modeldir + "mg_fe_binned.csv")
        ah = pd.read_csv(modeldir + "mg_h_binned.csv")
        labels = list(params.keys())
    
    
        return cls(params=params, labels=labels, afe=afe, ah=ah, samples=samples)
    

In [ ]:
result = MCMCResult.from_file("analytic_quad")

In [ ]:
from corner import corner

In [ ]:
def plot_corner(result, **kwargs):

    corner(result.samples[result.labels], labels=result.labels, **kwargs)
    return 

In [ ]:
def plot_samples_caah(mcmc_result, alpha=None, skip=10):
    ah = mcmc_result.ah
    labels = mcmc_result.labels
    samples = mcmc_result.samples[::skip]


    if alpha is None:
        alpha = 1 / len(samples)**(1/3) / 10

    for l, sample in samples.iterrows():
        y = np.sum([sample[label] * ah[label] for label in labels], axis=0)

        plt.plot(ah._x, gcem.abund_to_brak(y, "c") - ah._x, color="black", alpha=alpha)

    plt.errorbar(ah._x, gcem.abund_to_brak(ah.obs, "c") - ah._x, yerr=ah.obs_err / ah.obs / np.log(10), fmt="o", color=arya.COLORS[1])
    
    plt.xlabel("[Mg/H]")
    plt.ylabel("[C/Mg]")

In [ ]:
def plot_samples_caafe(mcmc_result, alpha=None, skip=10):
    ah = mcmc_result.afe
    labels = mcmc_result.labels
    samples = mcmc_result.samples[::skip]


    if alpha is None:
        alpha = 1 / len(samples)**(1/3) / 10

    for l, sample in samples.iterrows():
        y = np.sum([sample[label] * ah[label] for label in labels], axis=0)

        plt.plot(ah._x, gcem.abund_to_brak(y, "c"), color="black", alpha=alpha)

    plt.errorbar(ah._x, gcem.abund_to_brak(ah.obs, "c"), yerr=ah.obs_err / ah.obs / np.log(10), fmt="o", color=arya.COLORS[1])
    
    

In [ ]:
plot_corner(result)

In [ ]:
plot_samples_caah(result)

In [ ]:
plot_samples_caafe(result)

In [ ]:
yc = surp.yield_models.BiLogLin_CC(y0=0.000, zeta=0.001, y1=-0.002)

In [ ]:
x = np.linspace(-3, 2, 10000)

plt.plot(x, [yc(0.016 * 10**xx) for xx in x])